### Import libraries

In [51]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import shutil
import matplotlib.image as mpimg
import glob
import os

## Prepare the data

In [52]:
# Model / data parameters
img_width, img_height = 28, 28
epochs = 50
batch_size = 16
input_shape = (28, 28, 1)

notMNIST_load_data is a function that loads the notMNIST dataset. It returns three tuples containing the training, validation and test data. Each tuple is formed by a numpy array containing the images and a numpy array containing the labels (0 to 9).

In [55]:
def notMNIST_load_data() : 
    data_dir_letters = 'data/notMNIST_small'

    x_train = []
    y_train = []
    x_test = []
    y_test = []

    # we get the number of samples in the smallest class
    min_nb_samples = float('inf')
    for letter in os.listdir(data_dir_letters):
        nb_samples = len(os.listdir(os.path.join(data_dir_letters, letter)))
        min_nb_samples = min(min_nb_samples, nb_samples)

    # 80% of the data is used for training
    # 10% for validation
    # 10% for testing
    nb_train_samples = int(min_nb_samples * 0.8)
    # nb_validation_samples = int(min_nb_samples * 0.1)
    nb_validation_samples = 0
    nb_test_samples = nb_samples - nb_train_samples - nb_validation_samples

    # for each letter folder, we copy the images in the train, validation or test tuple and the label
    # TODO : randomize the order of the images would be better ?
    for letter in os.listdir(data_dir_letters):
        index = 0

        for image in glob.iglob(os.path.join(data_dir_letters, letter, "*.png")):

            if index < nb_train_samples:
                pixels_array = mpimg.imread(image)
                x_train.append(pixels_array)
                y_train.append(ord(letter) - 65)
            #elif index < nb_train_samples + nb_validation_samples:
                
            elif index < nb_train_samples + nb_validation_samples + nb_test_samples:
                pixels_array = mpimg.imread(image)
                x_test.append(pixels_array)
                y_test.append(ord(letter) - 65)
            index += 1
    
    return (np.array(x_train), np.array(y_train)), (np.array(x_test), np.array(y_test))